In [2]:
import pandas as pd
import requests as r
import sqlalchemy
import time
pd.options.display.max_columns = None

In [5]:
user_agent={"User-agent":"super-data-getter"}
after=""
data=[]
url="https://api.covidactnow.org/v2/states.json?apiKey=beba7a7eda8f44f99db168588523cf3f"

In [27]:
response = r.get(url, headers=user_agent).json()

In [28]:
response

[{'fips': '02',
  'country': 'US',
  'state': 'AK',
  'county': None,
  'level': 'state',
  'lat': None,
  'locationId': 'iso1:us#iso2:us-ak',
  'long': None,
  'population': 731545,
  'metrics': {'testPositivityRatio': 0.013767783386874713,
   'testPositivityRatioDetails': {'source': 'other'},
   'caseDensity': 9.393036069453787,
   'contactTracerCapacityRatio': 0.683991683991684,
   'infectionRate': 0.806033113721,
   'infectionRateCI90': 0.16758538635200004,
   'icuHeadroomRatio': 0.1388888888888889,
   'icuHeadroomDetails': {'currentIcuCovid': 5,
    'currentIcuCovidMethod': 'actual',
    'currentIcuNonCovid': 95,
    'currentIcuNonCovidMethod': 'actual'},
   'icuCapacityRatio': 0.7633587786259542,
   'vaccinationsInitiatedRatio': 0.44271097471789156,
   'vaccinationsCompletedRatio': 0.3804345597331675},
  'riskLevels': {'overall': 1,
   'testPositivityRatio': 0,
   'caseDensity': 1,
   'contactTracerCapacityRatio': 1,
   'infectionRate': 0,
   'icuHeadroomRatio': 0,
   'icuCapacit

In [45]:
cases=[]
deaths=[]
positiveTests=[]
newCases=[]
vaccinesDistributed=[]
vaccinationsInitiated=[]
vaccinationsCompleted=[]
vaccinesAdministered=[]


for states in range(0,len(response)):
    cases.append(response[states]['actuals']['cases'])
    deaths.append(response[states]['actuals']['deaths'])
    positiveTests.append(response[states]['actuals']['positiveTests'])
    newCases.append(response[states]['actuals']['newCases'])
    vaccinesDistributed.append(response[states]['actuals']['vaccinesDistributed'])
    vaccinationsInitiated.append(response[states]['actuals']['vaccinationsInitiated'])
    vaccinationsCompleted.append(response[states]['actuals']['vaccinationsCompleted'])
    vaccinesAdministered.append(response[states]['actuals']['vaccinesAdministered'])

In [54]:
df_data=pd.DataFrame()

df_data['cases']=cases
df_data['deaths']=deaths
df_data['positiveTests']=positiveTests
df_data['newCases']=newCases
df_data['vaccinesDistributed']=vaccinesDistributed
df_data['vaccinationsInitiated']=vaccinationsInitiated
df_data['vaccinationsCompleted']=vaccinationsCompleted
df_data['vaccinesAdministered']=vaccinesAdministered

df_data.head()

,cases,deaths,positiveTests,newCases,vaccinesDistributed,vaccinationsInitiated,vaccinationsCompleted,vaccinesAdministered
0,69186,352,111234,98,797805,323863,278305,590878
1,530330,11045,536733,1144,4393980,1709761,1362834,2906102
2,339406,5805,234333,244,2721900,1142312,890328,1981006
3,873264,17497,875469,540,7538145,3212922,2470567,5663654
4,3772740,62759,3723947,1587,45321600,21273160,15541344,35767717


In [74]:
state=[]

for i in range(0,len(response)):
    state.append(response[i]['state'])

In [78]:
state_df=pd.DataFrame()
state_df['state']=state
state_df.head()

,state
0,AK
1,AL
2,AR
3,AZ
4,CA


In [86]:
df=pd.concat([state_df, df_data], axis=1)
df.head()

,state,cases,deaths,positiveTests,newCases,vaccinesDistributed,vaccinationsInitiated,vaccinationsCompleted,vaccinesAdministered
0,AK,69186,352,111234,98,797805,323863,278305,590878
1,AL,530330,11045,536733,1144,4393980,1709761,1362834,2906102
2,AR,339406,5805,234333,244,2721900,1142312,890328,1981006
3,AZ,873264,17497,875469,540,7538145,3212922,2470567,5663654
4,CA,3772740,62759,3723947,1587,45321600,21273160,15541344,35767717


In [79]:
coord_file=pd.read_csv('states_coordinates.csv')

In [85]:
lat=coord_file[['latitude']]
long=coord_file[['longitude']]

df['lat']=lat
df['long']=long

df.head()

,state,cases,deaths,positiveTests,newCases,vaccinesDistributed,vaccinationsInitiated,vaccinationsCompleted,vaccinesAdministered,lat,long
0,AK,69186,352,111234,98,797805,323863,278305,590878,63.588753,-154.493062
1,AL,530330,11045,536733,1144,4393980,1709761,1362834,2906102,32.318231,-86.902298
2,AR,339406,5805,234333,244,2721900,1142312,890328,1981006,35.201050,-91.831833
3,AZ,873264,17497,875469,540,7538145,3212922,2470567,5663654,34.048928,-111.093731
4,CA,3772740,62759,3723947,1587,45321600,21273160,15541344,35767717,36.778261,-119.417932


## Loader
here we make a conection to sqllite db 

In [15]:
engine=sqlalchemy.create_engine('sqlite:///covid.db')

In [21]:
# table_exists = engine.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='reddit_posts';")
# if len([x for x in table_exists])>0:
#     print('we have a table')
#     d= engine.execute("SELECT name FROM reddit_posts")
#     existing_names = [x[0] for x in d]
#     rows_to_insert=df.loc[~df['name'].isin(existing_names)]
#     if not rows_to_insert.empty:
#         print('we have some new rows')
#         df.to_sql("reddit_posts", index=False, if_exists="append" ,con=engine)
        
        
    
# else:
#     df.to_sql("reddit_posts", index=False, con=engine)

we have a table


In [22]:
# table_exists = engine.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='reddit_comments';")
# if len([x for x in table_exists])>0:
#     d = engine.execute(f'''select link, text from reddit_comments where link in ('{f"','".join(list(comments_df['link'].unique()))}')''')
#     links=[]
#     texts=[]
#     for obj in d:
#         links.append(obj[0])
#         texts.append(obj[1])
#     rows_to_insert = comments_df.loc[ ~(
#                 (comments_df['link'].isin(links)) 
#                 & (comments_df['text'].isin(texts))
#                )
#            ]
#     if not rows_to_insert.empty:
#         print('we have some new rows')
#         rows_to_insert.to_sql("reddit_comments", index=False, if_exists="append" ,con=engine)
        
        
# else:
#     comments_df.to_sql("reddit_comments",index=False,con=engine)

## verify results

In [25]:
pd.read_sql(''' select reddit_posts.permalink, reddit_comments.text from reddit_posts 
                join reddit_comments  on reddit_posts.permalink = reddit_comments.link
                where reddit_posts.permalink = '/r/CryptoCurrency/comments/mr8u6d/coinrule_trading_bot_buy_crypto_with_paypal/';
                ''',con=engine)


,permalink,text
0,/r/CryptoCurrency/comments/mr8u6d/coinrule_tra...,Do not buy through PayPal! You cannot remove o...
1,/r/CryptoCurrency/comments/mr8u6d/coinrule_tra...,Do you know what is the bot strategy to buy/sell?
2,/r/CryptoCurrency/comments/mr8u6d/coinrule_tra...,Honestly just DCA into strong projects and you...
3,/r/CryptoCurrency/comments/mr8u6d/coinrule_tra...,I actually haven’t seen an exchange that will ...
4,/r/CryptoCurrency/comments/mr8u6d/coinrule_tra...,I mean I want to pay with PayPal for bitcoin.
5,/r/CryptoCurrency/comments/mr8u6d/coinrule_tra...,"I wanna buy BTC and pay with PayPal, not use P..."
6,/r/CryptoCurrency/comments/mr8u6d/coinrule_tra...,"It has multiple, will send once home"
7,/r/CryptoCurrency/comments/mr8u6d/coinrule_tra...,Ok thanks
8,/r/CryptoCurrency/comments/mr8u6d/coinrule_tra...,"The moment someone releases a successful bot, ..."
9,/r/CryptoCurrency/comments/mr8u6d/coinrule_tra...,There's more money to invest
